# Imports

In [ ]:
import sys
import os
import json
import warnings
import pickle
from dotenv import load_dotenv

!pip install -U google-generativeai
import google.generativeai as genai

from google.colab import drive
from google.colab import files

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!pip install papermill
!pip install nbconvert
!pip install nbformat
!pip install IPython

import papermill as pm
import nbformat
from nbconvert import HTMLExporter
from IPython.display import HTML, display

warnings.filterwarnings("ignore")
%matplotlib inline

# Bootstrap

In [ ]:
np.random.seed(31071967)

load_dotenv()

drive.mount('/content/drive')

with open(f"{os.getenv('PROJECT_PATH')}/src/config.json", 'r') as f:
    project_config = json.load(f)
    project_config.pop('_comment', None)
    project_config.pop('_note', None)
    f.close()

In [ ]:
# Define our GOOGLE API KEY  in .env file. GOOGLE_API_KEY=<your api key>
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
def get_best_available_model():
    """
    FORCE uses the specific stable version 'gemini-1.5-flash'.
    We avoid 'listing' models because that finds restricted experimental versions.
    """
    # This is the exact name of the stable, free-tier friendly model.
    # We return it directly to stop the code from picking "2.5" or "latest".
    return "gemini-1.5-flash"

In [ ]:
def extract_prediction_params(user_sentence):
    # 1. Dynamically get the model name
    model_name = get_best_available_model()

    if not model_name:
        print("CRITICAL ERROR: No valid Gemini models found for your API key.")
        return None

    print(f"[System] Using model: {model_name}")
    model = genai.GenerativeModel(model_name)

    prompt = f"""
    You are a data extraction assistant.
    Analyze this user request: "{user_sentence}"

    Extract:
    1. "stock_ticker": The ticker symbol (e.g., "Apple" -> "AAPL").
    2. "days": The integer number of days for prediction (e.g., "2 weeks" -> 14).

    Return ONLY raw JSON. Keys: 'stock_ticker', 'days'.
    """

    try:
        # Note: We removed 'response_mime_type' just in case your library version is old.
        # We will handle the JSON manually.
        response = model.generate_content(prompt)

        # Clean up the text to ensure it's valid JSON (remove markdown backticks)
        clean_text = response.text.replace("```json", "").replace("```", "").strip()
        data = json.loads(clean_text)
        return data

    except Exception as e:
        print(f"Error extracting data: {e}")
        return None

In [ ]:
def get_target_tkl_from_user():

  user_input = input("What would you like to predict and for how long: ") # e.g. "Predict Google for 2 weeks"
  result = extract_prediction_params(user_input)

  while not result or result == None or result.get('stock_ticker') == None:
    print("\nFailed to understand the proment. Please try again.")
    user_input = input("\nRequest: ")
    result = extract_prediction_params(user_input)

  return result

In [ ]:
def update_param_in_projet_config_file(param, new_value, file_path=f"{os.getenv('PROJECT_PATH')}/src/config.json"):

    # 1. Check if file exists, if not, create a placeholder
    if not os.path.exists(file_path):
        print(f"[Warning] {file_path} not found. Creating a new one.")
        data = {}
    else:
        # 2. Read the existing data
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
        except json.JSONDecodeError:
            print(f"[Error] {file_path} is empty or corrupted. Starting fresh.")
            data = {}

    # 3. Update the specific variable
    data[param] = new_value

    # 4. Write everything back to the file
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4) # indent=4 makes it readable for humans

    print(f"Successfully updated {param} to: {new_value}")

In [ ]:
def run_notebook(notebook_name, output_name, parameters=None):

  # --- Execute the proviuse notebook with parameters ---
  pm.execute_notebook(
      input_path = notebook_name,
      output_path = output_name,
      log_output=False,  # don't print logs while running
      progress_bar=True
  )

  # --- Convert the executed notebook to HTML ---
  nb = nbformat.read(output_name, as_version=4)
  html_exporter = HTMLExporter()
  html_exporter.template_name = "lab"  # modern look; alternatives: 'classic', 'basic'
  body, _ = html_exporter.from_notebook_node(nb)

  # --- Display the HTML result inline ---
  display(HTML(body))

In [ ]:
def predict(user_selected_tkl, user_selected_days):

  project_config['TKL'] = user_selected_tkl
  update_param_in_projet_config_file('TKL', user_selected_tkl)
  project_config['PRED.DAYS'] = user_selected_days
  update_param_in_projet_config_file('PRED.DAYS', user_selected_days)

  input_file = f"{os.getenv('PROJECT_PATH')}{project_config['notebooks_directory']}{project_config['notebook9']}"
  output_file = f"{os.getenv('PROJECT_PATH')}{project_config['output_directory']}{project_config['TKL']}.{project_config['output9']}"

  run_notebook(input_file, output_file)

In [33]:
from google.generativeai.client import configure
print("--- Stock Prediction Setup ---")

confirmed_user_params = False
while not confirmed_user_params:

  result = get_target_tkl_from_user()
  days = int(result.get('days')) if result.get('days') != None else int(project_config["PRED.DAYS"])

  print(f"Ticker: {result.get('stock_ticker')}")
  print(f"Days:   {days}")

  if days < 1 or days > 10:
    print("Days should be between 1 and 10")
    continue

  confirmed_user_params = (input("Pls confirm: [y=yes, n=no] ") == 'y')

predict(result.get('stock_ticker'), days)

--- Stock Prediction Setup ---
What would you like to predict and for how long: נוידיאה
[System] Scanning for available models...
[System] Using model: models/gemini-pro-latest


Error extracting data: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
Please retry in 47.70035384s.

Failed to under

Error extracting data: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
Please retry in 47.149893306s.

Failed to unde

KeyboardInterrupt: Interrupted by user

In [39]:
import google.generativeai as genai
import json
import time

# --- CONFIGURATION ---
# genai.configure(api_key="YOUR_API_KEY")

def get_working_model_name():
    """
    Based on your specific log, we prioritize 'Lite' models
    which typically have better availability than 'Preview' models.
    """
    # We search specifically for the model we saw in your log
    target_models = [
        "models/gemini-2.0-flash-lite",      # Best bet for speed/quota
        "models/gemini-2.0-flash-lite-001",
        "models/gemini-2.0-flash",           # Second best
        "models/gemini-2.5-flash-lite"       # Alternative
    ]

    available = [m.name for m in genai.list_models()]

    # Pick the first matching target
    for target in target_models:
        if target in available:
            return target

    # Fallback to whatever flash exists if Lite is missing
    for m in available:
        if "flash" in m and "2.5" not in m: # Avoid 2.5 if possible
            return m

    return available[0] # Absolute fallback

def extract_prediction_params(user_sentence):
    model_name = get_working_model_name()
    print(f"[System] Forced Safe Model: {model_name}")

    model = genai.GenerativeModel(model_name)

    prompt = f"""
    Extract data from: "{user_sentence}"
    Return JSON with keys:
    1. "stock_ticker": The English ticker symbol (e.g. "נוידיאה" -> "NVDA").
    2. "days": Integer number of days.
    """

    for attempt in range(3):
        try:
            # We add a tiny delay to be safe
            time.sleep(1)
            response = model.generate_content(prompt)
            text = response.text.replace("```json", "").replace("```", "").strip()
            return json.loads(text)
        except Exception as e:
            if "429" in str(e):
                print(f"[Warning] 429 Rate Limit. Waiting 5s...")
                time.sleep(5)
            else:
                print(f"[Error] {e}")
                return None
    return None

# --- Main Program ---
if __name__ == "__main__":
    print("--- Stock Prediction Setup (Lite Version) ---")
    user_input = input("What would you like to predict and for how long: ")

    result = extract_prediction_params(user_input)

    if result:
        print(f"\n--- Success ---")
        print(f"Ticker: {result.get('stock_ticker')}")
        print(f"Days:   {result.get('days')}")
    else:
        print("Failed to extract data.")

--- Stock Prediction Setup (Lite Version) ---
What would you like to predict and for how long: נוידיאה
[System] Forced Safe Model: models/gemini-2.0-flash-lite


[Warning] 429 Rate Limit. Waiting 5s...


[Warning] 429 Rate Limit. Waiting 5s...


[Warning] 429 Rate Limit. Waiting 5s...
Failed to extract data.
